In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import photogrammetry as pg

# %matplotlib qt

plt.style.use('default')

%load_ext autoreload
%autoreload 2

In [ ]:
# in case a new board type is needed
# pg.generate_charuco_board(pg.BOARD_SQUARE_SIZE, pg.BOARD_ARUCO_SIZE, aruco_dict=cv2.aruco.DICT_4X4_1000, gen_png=True)

In [ ]:
%autoreload 2
# check for pickled camera matrices to avoid expensive recalibration
run = '20230616_0'
cal_path = f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/calibration/06182023_1'

extensions = ['.jpeg', '.jpg', '.JPG', '.PNG']#, '.tiff', '.TIFF']
files = []
for ext in extensions:
    files += sorted(glob.glob(os.path.join(cal_path, '**' + ext), recursive=False))
assert files, 'No image files found when searching for images for camera calibration.'

if not (
    os.path.exists(os.path.join(cal_path, 'camera_cal_mtx.pickle')) and
    os.path.exists(os.path.join(cal_path, 'camera_cal_dist.pickle')) and
    os.path.exists(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'))
):
    # calibrate the camera for distortion
    mtx, dist, optimal_camera_matrix, roi = pg.calibrate_camera(
        files,
        square_size=pg.BOARD_SQUARE_SIZE,
        aruco_size=pg.BOARD_ARUCO_SIZE,
        guess_intrinsics=True,
        plot=False
    )

    with open(os.path.join(cal_path, 'camera_cal_mtx.pickle'), 'wb') as f:
        pickle.dump(mtx, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(cal_path, 'camera_cal_dist.pickle'), 'wb') as f:
        pickle.dump(dist, f, protocol=pickle.HIGHEST_PROTOCOL)
    with open(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'), 'wb') as f:
        pickle.dump(optimal_camera_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open(os.path.join(cal_path, 'camera_cal_mtx.pickle'), 'rb') as f:
        mtx = pickle.load(f)
    with open(os.path.join(cal_path, 'camera_cal_dist.pickle'), 'rb') as f:
        dist = pickle.load(f)
    with open(os.path.join(cal_path, 'camera_cal_optimal_camera_matrix.pickle'), 'rb') as f:
        optimal_camera_matrix = pickle.load(f)
print(mtx)
print(dist)
print(optimal_camera_matrix)
print(cv2.calibrationMatrixValues(mtx, (4032, 3024), 1.7e-6 * 4032, 1.7e-6 * 3024))

`sort_images_exif ./ ./`

In [ ]:
meas_path = f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/measurement/'
files = []
for ext in extensions:
    files += sorted(glob.glob(os.path.join(meas_path, '**' + ext)))
assert files, 'No image files found when searching for images for measurement.'

In [ ]:
# use camera cal matrix to de-distort a few images to check
for file in files[:2]:
    fig, ax = plt.subplots()
    ax.imshow(pg.load_to_gray(file, mtx, dist), cmap='bone')

In [ ]:
origin_id = 48
raft_id = 50

In [ ]:
img_data = pg.measure_images(files, mtx, dist, aruco_ids=[origin_id, raft_id], plot=False)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5), subplot_kw={'projection':'3d'})
ax.view_init(elev=0, azim=-90, roll=0)

for file, entities in img_data.items():
    for id, pose in entities.items():
        for key, val in pose.items():
            print(os.path.basename(file), id, key, val)
            if key == 'tvec':
                if id == raft_id:
                    color = 'k'
                elif id == origin_id:
                    color = 'b'
                elif id == 'board':
                    color = 'r'
                else:
                    continue
                ax.scatter(
                    val[0],
                    val[1],
                    val[2],
                    color=color,
                    # label=id,
                    alpha=0.7
                )
# ax.legend()
ax.set_box_aspect([1,1,1])
origin = np.array([-0.10, -0.00, 0.0])
sz = .3
ax.set_xlim(-sz + origin[0], sz + origin[0])
ax.set_ylim(-sz + origin[1], sz + origin[1])
ax.set_zlim(-sz + origin[2], sz + origin[2]);

In [ ]:
fig, ax = plt.subplots()
origin_zs = []
raft_zs = []
for file, entities in img_data.items():
    for id, pose in entities.items():
        for key, val in pose.items():
            if key == 'tvec':
                if id == origin_id:
                    origin_zs.append(val[2])
                elif id == raft_id:
                    raft_zs.append(val[2])
                else:
                    continue
origin_zs = np.array(origin_zs)
raft_zs = np.array(raft_zs)
inliers = np.where(np.abs(origin_zs - np.nanmean(origin_zs)) <= np.nanmean(origin_zs) + 5. * np.nanstd(origin_zs))[0]
ax.plot(range(len(origin_zs[inliers])), origin_zs[inliers]*1e3, marker='o', linestyle='none')
ax.plot(range(len(raft_zs[inliers])), raft_zs[inliers]*1e3, marker='o', linestyle='none')
print('origin std', np.nanstd(origin_zs[inliers]))
print('raft std', np.nanstd(raft_zs[inliers]))
ax.set_xlabel('Scan Number')
ax.set_ylabel('z-position in board frame (mm)')
print(raft_zs[inliers])

In [ ]:
thetas = []
for file in files:
    if file in img_data.keys():
        try:
            theta = pg.estimate_angular_offset(img_data, file, 'board', raft_id)
            thetas.append(theta)
        except:
            pass
thetas = np.array(thetas)

fig, ax = plt.subplots()
ax.hist(thetas, bins=50);

In [ ]:
%autoreload

commanded_pts, actual_points, residuals = pg.post_process_scan(
    img_data,
    f'/home/evanmayer/TIME_data/mirror_mapping/testing/{run}/results/',
    os.path.join('..', 'data', 'input', 'profiles', '25in_breadboard_raster_skipline_10x10_0.30mx0.30m.csv'),
    raft_id,
    origin_id=None,#origin_id,
    savefig=False
)